In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/binary-classification-with-a-bank-dataset-clone/sample_submission.csv
/kaggle/input/binary-classification-with-a-bank-dataset-clone/train.csv
/kaggle/input/binary-classification-with-a-bank-dataset-clone/test.csv


In [2]:
# ==========================================================
# 🚀 Kaggle Playground 2025 - Bank Term Deposit Prediction
# ==========================================================

import numpy as np
import pandas as pd
import lightgbm as lgb

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

# =========================
# 📥 Load Data
# =========================
train = pd.read_csv("/kaggle/input/binary-classification-with-a-bank-dataset-clone/train.csv")
test = pd.read_csv("/kaggle/input/binary-classification-with-a-bank-dataset-clone/test.csv")

print("Train shape:", train.shape)
print("Test shape:", test.shape)

# =========================
# 🎯 Target + ID
# =========================
y = train["y"]
train_ids = train["id"]
test_ids = test["id"]

X = train.drop(columns=["y", "id"])
X_test = test.drop(columns=["id"])

# =========================
# 🔍 Handle Categorical Features
# =========================
cat_cols = X.select_dtypes(include=["object"]).columns.tolist()

for col in cat_cols:
    X[col] = X[col].astype("category")
    X_test[col] = X_test[col].astype("category")

# =========================
# 🔁 Stratified 5-Fold CV
# =========================
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

oof_preds = np.zeros(len(X))
test_preds = np.zeros(len(X_test))

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    
    print(f"\n===== Fold {fold+1} =====")
    
    X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    model = lgb.LGBMClassifier(
        n_estimators=2000,
        learning_rate=0.03,
        max_depth=-1,
        num_leaves=64,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    )
    
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        eval_metric="auc",
        callbacks=[lgb.early_stopping(200)]
    )
    
    oof_preds[val_idx] = model.predict_proba(X_val)[:, 1]
    test_preds += model.predict_proba(X_test)[:, 1] / skf.n_splits
    
    fold_auc = roc_auc_score(y_val, oof_preds[val_idx])
    print(f"Fold {fold+1} AUC: {fold_auc:.6f}")

# =========================
# 📊 Final CV Score
# =========================
overall_auc = roc_auc_score(y, oof_preds)
print("\nOverall CV AUC:", overall_auc)

# =========================
# 📝 Submission File
# =========================
submission = pd.DataFrame({
    "id": test_ids,
    "y": test_preds
})

submission.to_csv("submission.csv", index=False)

print("\n✅ submission.csv created successfully")


Train shape: (750000, 18)
Test shape: (250000, 17)

===== Fold 1 =====
[LightGBM] [Info] Number of positive: 72391, number of negative: 527609
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.035686 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1009
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.120652 -> initscore=-1.986273
[LightGBM] [Info] Start training from score -1.986273
Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[1876]	valid_0's auc: 0.969455	valid_0's binary_logloss: 0.141296
Fold 1 AUC: 0.969455

===== Fold 2 =====
[LightGBM] [Info] Number of positive: 72391, number of negative: 527609
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testin